In [1]:
import numpy as np
import random
from IPython.display import Image, display 
from graphviz import Digraph

# Define constants, variables, and operators
constant_range = np.linspace(0.5, 10, num=40)
CONSTANTS = list(constant_range)

def safe_divide(a, b):
    return np.divide(a, b, where=b != 0, out=np.full_like(a, np.nan)) # Safe divide for b != 0, else nan. With NaN, you'll know exactly where and why a result is invalid. With 0, you'll have valid numbers for calculations, but the invalid results may not be easy to track.

def safe_sqrt(a):
    return np.sqrt(a, where=a >= 0, out=np.full_like(a, np.nan)) # Safe sqrt for non-negative a, else nan. 

def safe_log(a):
    return np.log(a, where=a > 0, out=np.full_like(a, -np.inf)) # Safe log for positive a, else -inf

OPERATORS = {
    'add': np.add,
    'sub': np.subtract,
    'mul': np.multiply,
    'div': safe_divide,
    'sin': np.sin,
    'cos': np.cos,
    'tan': np.tan,
    'sqrt': safe_sqrt,
    'log': safe_log,
}

Node class

In [2]:
class Node:
    def __init__(self, value, is_operator=False):
        """
        Initialize a Node.
        
        :param value: The value of the node (operator, constant, or variable name).
        :param is_operator: True if the node represents an operator, otherwise False.
        """
        self.value = value
        self.is_operator = is_operator
        self.left = None
        self.right = None

    def evaluate(self, variables):
        """
        Evaluate the subtree rooted at this node.
        
        :param variables: A dictionary mapping variable names to their values.
        :return: The result of evaluating the subtree.
        """
        # Check for None values to avoid errors
        if self is None:
            return 0

        if self.is_operator:
            # Unary operator (e.g., sin, log, etc.)
            #if self.value in ['sin', 'cos', 'tan', 'sqrt', 'log', 'log10', 'sinh', 'cosh', 'tanh', 'exp', 'square', 'abs']:
            if self.value in ['sin', 'cos', 'tan', 'sqrt', 'log']:    
                if self.left is None:
                    raise ValueError(f"Missing left child for operator {self.value}")
                left_value = self.left.evaluate(variables)
                return OPERATORS[self.value](left_value)
            
            # Binary operator (e.g., add, sub, mul, div, etc.)
            #elif self.value in ['add', 'sub', 'mul', 'div', 'pow']:
            elif self.value in ['add', 'sub', 'mul', 'div']:
                if self.left is None or self.right is None:
                    raise ValueError(f"Missing children for operator {self.value}")
                left_value = self.left.evaluate(variables)
                right_value = self.right.evaluate(variables)
                return OPERATORS[self.value](left_value, right_value)
        
        elif self.value in variables:
            # Variable node (e.g., x0, x1, etc.)
            return variables[self.value]
        
        # Constant node (just return the constant value)
        return self.value

Individual class

In [20]:
class Individual:
    def __init__(self, root, file_path):
        """
        Initialize an Individual (a tree).
        
        :param root: The root node of the tree.
        """
        self.root = root
        self.fitness_value = None
        
        if file_path:
            self.fitness_value = self.fitness(file_path)

    def evaluate(self, variables):
        """
        Evaluate the tree.
        
        :param variables: A dictionary mapping variable names to their values.
        :return: The result of evaluating the tree.
        """
        return self.root.evaluate(variables)
    
    def tree_size(self):
        """
        Return the number of nodes in the tree.
        """
        def count_nodes(node):
            if node is None:
                return 0
            return 1 + count_nodes(node.left) + count_nodes(node.right)
        
        return count_nodes(self.root)

    def fitness(self, file_path):
        """
        Calculate the fitness of the individual using Mean Squared Error (MSE).
        
        :param file_path: Path to the .npz file containing 'x' and 'y'.
        :return: The MSE between the predicted and actual y values.
        """
        # Load the data
        data = np.load(file_path)
        x = data['x']
        y = data['y']

        # Initialize variables for prediction
        num_features = x.shape[0]
        variables = {f'x{i}': None for i in range(num_features)}

        # Compute predictions
        y_pred = []
        for i in range(x.shape[1]):  # Iterate over each column
            for j in range(num_features):  # Set variable values for this row
                variables[f'x{j}'] = x[j, i]
            y_pred.append(self.evaluate(variables))
        y_pred = np.array(y_pred)

        # Calculate MSE
        mse = np.mean((y - y_pred) ** 2)
        return mse
    
    #variant of fitness involving complexity penalty
    def fitness2(self, file_path, complexity_penalty=0.01):
        data = np.load(file_path)
        x = data['x']
        y = data['y']
        num_features = x.shape[0]
        variables = {f'x{i}': None for i in range(num_features)}

        y_pred = []
        for i in range(x.shape[1]):
            for j in range(num_features):
                variables[f'x{j}'] = x[j, i]
            y_pred.append(self.evaluate(variables))
        y_pred = np.array(y_pred)

        mse = np.mean((y - y_pred) ** 2)
        return mse + complexity_penalty * self.tree_size()

    def get_all_nodes(self):
        """
        Retrieve all nodes in the tree.

        :return: A list of all nodes in the tree.
        """
        def collect_nodes(node):
            if node is None:
                return []
            return [node] + collect_nodes(node.left) + collect_nodes(node.right)
        
        return collect_nodes(self.root)
    

    def __str__(self):
        """
        Return a string representation of the tree.
        """
        return self._str_helper(self.root)

    def _str_helper(self, node):
        """
        Helper function for string representation. Recursively traverse the tree.
        """
        if node is None:
            return ""
        
        if node.is_operator:
            if node.right is not None:
                return f"({node.value}({self._str_helper(node.left)}, {self._str_helper(node.right)})"
            else:
                return f"{node.value}({self._str_helper(node.left)})"
        return str(node.value)

In [4]:
def mse_percentage(mse_value, file_path):
        """
        Calculate the MSE percentage to assess how logical the fitness value is.

        :param file_path: Path to the .npz file containing 'x' and 'y'.
        :return: The MSE percentage.
        """
        
        # Load the data to get the y values for min and max
        data = np.load(file_path)
        y = data['y']
        
        # Calculate the range of y values
        y_max = np.max(y)
        print(y_max)
        y_min = np.min(y)
        print(y_min)
        
        # Calculate the MSE percentage
        mse_percentage = (mse_value / (y_max - y_min)) * 100
        
        return mse_percentage

Create tree

In [21]:
# Create a random tree out of the class Individual
def create_tree(num_nodes, file_name):
    """
    Create a random tree with the specified number of nodes.
    
    :param num_nodes: The number of nodes in the tree.
    :param file_name:  .npz file.
    :return: A random tree as an Individual object.
    """
    # Assuming x_data is loaded from 'problem_20.npz', let's load it properly
    data = np.load(file_name)
    x_data = data['x']  # Assuming 'x' is the key for your data in the .npz file
    y_data = data['y']  # Assuming 'y' is the key for your labels (if you need it)

    # Make sure x_data is a NumPy array
    x_data = np.array(x_data)

    num_vars = x_data.shape[0]  # Number of variables (rows in x)
    VARIABLES = [f'x{i}' for i in range(num_vars)]  # Generate the variable names dynamically
    variable_included = False  # Flag to check if at least one variable is included in the tree

    def random_node(is_operator=False):
        """Helper function to generate a random node (operator or constant/variable)."""
        nonlocal variable_included
        if is_operator:
            # Random operator node (should never be a terminal node)
            value = random.choice(list(OPERATORS.keys()))
            return Node(value, is_operator=True)
        else:
            # Random constant or variable node (only for leaf nodes)
            value = random.choice(CONSTANTS) if random.random() < 0.1 else random.choice(VARIABLES)
            if value in VARIABLES:
                variable_included = True
            return Node(value, is_operator=False)

    def build_tree(nodes_left, is_root=True):
        """Recursively build a tree. Ensure the first node is not a terminal node."""
        if nodes_left == 1:
            # If only one node is left, it should be a terminal node (constant or variable)
            return random_node(is_operator=False), 1
        else:
            # Choose an operator (either unary or binary)
            operator = random.choice(list(OPERATORS.keys()))
            node = Node(operator, is_operator=True)

            if operator in ['sin', 'cos', 'tan', 'sqrt', 'log']:
            #if operator in ['sin', 'cos', 'tan', 'sqrt', 'log', 'log10', 'sinh', 'cosh', 'tanh', 'exp', 'square', 'abs']:
                # Unary operator: Only one child is required
                right_subtree_size = nodes_left - 1
                right_child, _ = build_tree(right_subtree_size, is_root=False)
                node.left = right_child
                return node, 1 + right_subtree_size  # 1 for the operator, + subtree size
            elif operator in ['add', 'sub', 'mul', 'div']:
            #elif operator in ['add', 'sub', 'mul', 'div', 'pow']:
                # Binary operator: Two children are required
                if nodes_left <= 2:
                    # If there are only two nodes left, force it to be a terminal node
                    left_subtree_size = 1
                    right_subtree_size = 1
                else:
                    right_subtree_size = random.randint(1, nodes_left - 2)
                    left_subtree_size = nodes_left - 1 - right_subtree_size

                # Build left and right subtrees
                left_child, left_size = build_tree(left_subtree_size, is_root=False)
                right_child, right_size = build_tree(right_subtree_size, is_root=False)

                node.left = left_child
                node.right = right_child

                return node, 1 + left_size + right_size  # 1 for the operator, + left and right subtree sizes

    # Build the tree and return it as an Individual object
    root_node, _ = build_tree(num_nodes)
    
    # Ensure at least one variable is included
    if not variable_included:
        print("No variable included in the tree; retrying...")
        return create_tree(num_nodes, file_name)  # Retry tree creation if no variable was included
    
    return Individual(root_node, file_name)

In [23]:
def draw_tree(tree):
    dot = Digraph(comment='Tree')
    
    # Set to track unique nodes by their value, but we're not excluding repeats
    node_counter = {}  # This will help track how many times a node has been used
    
    def add_edges(node, parent=None):
        if node:
            # Create a unique identifier for the node based on its value and repetition count
            node_name = str(node.value)
            
            # Track how many times we've seen this node (for repeating nodes)
            if node_name not in node_counter:
                node_counter[node_name] = 0
            node_counter[node_name] += 1

            # Append the repetition count to make the node name unique
            unique_node_name = f"{node_name}_{node_counter[node_name]}"

            # Add node to the graph if it has not been added
            dot.node(unique_node_name, label=str(node.value))
            
            if parent:
                # Connect parent to this node
                dot.edge(str(parent), unique_node_name)

            # Recursively add left and right children
            if node.left:
                add_edges(node.left, unique_node_name)
            if node.right:
                add_edges(node.right, unique_node_name)

    # Start from the root of the tree
    add_edges(tree.root)

    # Render the graph to a file and view it
    #dot.render('tree_visualization')
    
    img_data = dot.pipe(format='png')
    display(Image(img_data))


Population class

In [11]:
class Population:
    def __init__(self, size, num_nodes, file_name, individuals=None):
        """
        Initialize a population of random individuals (trees).
        
        :param size: Number of individuals in the population.
        :param num_nodes: Number of nodes for each tree.
        :param file_name: the .npz file containing the data
        """
        # self.individuals = [
        #     create_tree(num_nodes, x_data) for _ in range(size)
        # ]
        
        # Assuming x_data is loaded from 'problem_20.npz', let's load it properly
        data = np.load(file_name)
        x_data = data['x']  # Assuming 'x' is the key for your data in the .npz file
        y_data = data['y']  # Assuming 'y' is the key for your labels (if you need it)

        # Make sure x_data is a NumPy array
        x_data = np.array(x_data)
        
        self.size = size
        
        if individuals:
            self.individuals = individuals
        else:
            self.individuals = []
            unique_individuals = set()
            
            while len(self.individuals) < size:
                individual = create_tree(num_nodes, file_name)
                fitness_value = individual.fitness(file_name)
                
                # Check individual is valid
                if np.isnan(fitness_value) or np.isinf(fitness_value):
                    continue
                
                tree_representation = str(individual)
                
                if tree_representation not in unique_individuals and individual.tree_size() == num_nodes:
                    self.individuals.append(individual) 
                    unique_individuals.add(tree_representation)
                

    def evaluate_fitness(self, file_path, complexity_penalty=0.01):
        """
        Evaluate the fitness of all individuals in the population.
        
        :param file_path: Path to the .npz file containing 'x' and 'y'.
        :param complexity_penalty: Penalty for tree complexity.
        """
        for individual in self.individuals:
            individual.fitness_value = individual.fitness(file_path)
        
    def get_best_individual(self):
        """
        Get the individual with the best fitness in the population.
        
        :return: The best individual.
        """
        return min(self.individuals, key=lambda ind: ind.fitness_value)
    
    def get_worst_individual(self):
        """
        Get the individual with the best fitness in the population.
        
        :return: The best individual.
        """
        return max(self.individuals, key=lambda ind: ind.fitness_value)

    def get_average_fitness(self):
        """
        Get the average fitness of the population.
        
        :return: Average fitness value.
        """
        return np.mean([ind.fitness_value for ind in self.individuals])
    
    def tournament_selection(self, tournament_size=10):
        """
        Perform probabilistic tournament selection to choose an individual from the population.
        
        :param tournament_size: Number of individuals in the tournament.
        :return: The selected individual.
        """
        # Randomly select individuals for the tournament
        tournament = random.sample(self.individuals, tournament_size)
        
        # Individual with best fitness
        winner = min(tournament, key=lambda ind: ind.fitness_value)

        return winner

In [26]:
def get_random_node(tree):
    """
    Select a random node from the tree.
    
    :return: A randomly selected node from the tree.
    """
    all_nodes = tree.get_all_nodes()
    return random.choice(all_nodes) if all_nodes else None

def rebuild_subtree(tree, node):
    """
    Rebuild a subtree starting from the given node, replicating it exactly.
    
    :param node: The root node of the subtree to replicate.
    :return: A new Node that is a copy of the subtree rooted at `node`.
    """
    if node is None:
        return None

    # Create a copy of the current node
    new_node = Node(value=node.value, is_operator=node.is_operator)

    # Recursively copy the left and right children
    new_node.left = rebuild_subtree(tree, node.left)
    new_node.right = rebuild_subtree(tree, node.right)

    return new_node

def extract_and_rebuild_subtree(tree, temperature=1.0):
    """
    Extract and rebuild a random subtree from the tree.
    
    :param temperature: Temperature parameter to control randomness (not yet used).
    :return: A new subtree that is a copy of the randomly chosen subtree.
    """
    # Get a random node from the tree
    #random_node = get_random_node(tree)
    random_node = get_random_node(tree, temperature)
    if random_node is None:
        raise ValueError("Tree is empty; cannot extract a subtree.")

    # Rebuild the subtree starting from the selected node
    subtree = Individual(rebuild_subtree(tree, random_node))
    return subtree


def replace_subtree(tree, target_node, new_subtree):
    """
    Replace a subtree in the tree with a new subtree.
    
    :param tree: The tree (Individual) to modify.
    :param target_node: The node where the replacement should happen.
    :param new_subtree: The new subtree to replace the old one.
    :return: The modified tree (Individual).
    """
    # Find the parent node of the target node
    parent_node = None
    stack = [tree.root]
    while stack:
        current_node = stack.pop()
        if current_node.left == target_node or current_node.right == target_node:
            parent_node = current_node
            break
        if current_node.left:
            stack.append(current_node.left)
        if current_node.right:
            stack.append(current_node.right)

    # Replace the target node with the new subtree
    if parent_node:
        if parent_node.left == target_node:
            parent_node.left = new_subtree
        else:
            parent_node.right = new_subtree
    else:
        # If the target node is the root, replace the entire tree
        tree.root = new_subtree

    return tree


def subtree_mutation(tree, file_name, temperature=1.0):
    """
    Perform subtree mutation on a given tree.
    
    :param tree: The tree (root node) to mutate.
    :param x_data: The x_data to use for generating new subtrees.
    :param temperature: A parameter controlling randomness (not yet used).
    """
    # Get a random node from the tree
    target_node = get_random_node(tree)
    
    if target_node is None:
        raise ValueError("Tree is empty; cannot perform mutation.")

    # Calculate the size of the subtree rooted at the selected node
    subtree_individual = Individual(target_node, file_name)
    subtree_size = subtree_individual.tree_size()

    # Create a new subtree with the same size
    new_subtree = create_tree(subtree_size, file_name).root

    # Replace the selected subtree with the new one
    replace_subtree(tree, target_node, new_subtree)
    
    return tree


def subtree_crossover(tree1, tree2, temperature=1.0):
    """
    Perform subtree crossover between two trees.
    
    :param tree1: The first tree (Individual) to crossover.
    :param tree2: The second tree (Individual) to crossover.
    :param x_data: The x_data to use for generating new subtrees.
    :param temperature: A parameter controlling randomness (not yet used).
    :return: Two new trees resulting from the crossover operation.
    """        
    node1 = get_random_node(tree1)
    node2 = get_random_node(tree2)
    
    if node1 is None or node2 is None:
        raise ValueError("One or both trees are empty; cannot perform crossover.")
    
    # Extract subtrees from each tree
    subtree1 = rebuild_subtree(tree1, node1)
    subtree2 = rebuild_subtree(tree2, node2)
    
    # Replace subtrees in the opposite trees
    new_tree1 = replace_subtree(tree1, node1, subtree2)
    new_tree2 = replace_subtree(tree2, node2, subtree1)
    
    return new_tree1, new_tree2

In [ ]:
#Assuming x_data is loaded from 'problem_20.npz', let's load it properly
data = np.load('../data/problem_0.npz')
x_data = data['x']  # Assuming 'x' is the key for your data in the .npz file
y_data = data['y']  # Assuming 'y' is the key for your labels (if you need it)

# Make sure x_data is a NumPy array
x_data = np.array(x_data)

# Create a random tree
random_tree = create_tree(10, '../data/problem_0.npz')
#draw_tree(random_tree)

print("Random Tree Structure:", random_tree)
print(random_tree.fitness('../data/problem_0.npz'))

# subtree = extract_and_rebuild_subtree(random_tree, temperature =1.0)
# draw_tree(subtree)

# new_subtree = create_tree(subtree.tree_size(), x_data)
# draw_tree(new_subtree)

#Perform subtree mutation
mutated = subtree_mutation(random_tree, file_name='../data/problem_0.npz')
#draw_tree(mutated)
print("Mutated Tree Structure:", mutated)
print(mutated.fitness('../data/problem_0.npz'))

In [ ]:
#Assuming x_data is loaded from 'problem_20.npz', let's load it properly
data = np.load('../data/problem_0.npz')

file_name = '../data/problem_0.npz'
x_data = data['x']  # Assuming 'x' is the key for your data in the .npz file
y_data = data['y']  # Assuming 'y' is the key for your labels (if you need it)

# Make sure x_data is a NumPy array
x_data = np.array(x_data)

# Create a random tree
random_tree1 = create_tree(10, file_name)
#draw_tree(random_tree1)
print(random_tree1)
random_tree2 = create_tree(10, file_name)
#draw_tree(random_tree2)
print(random_tree2)

new_tree1, new_tree2 = subtree_crossover(random_tree1, random_tree2)
#draw_tree(new_tree1)
#draw_tree(new_tree2)
print(new_tree1)
print(new_tree2)

START

In [ ]:
# Parameters
population_size = 50
tree_size = 10
max_tree_depth = 5
file_name = '../data/problem_0.npz'

# Assuming x_data is loaded from 'problem_20.npz', let's load it properly
data = np.load(file_name)
x_data = data['x']  # Assuming 'x' is the key for your data in the .npz file
y_data = data['y']  # Assuming 'y' is the key for your labels (if you need it)

# Make sure x_data is a NumPy array
x_data = np.array(x_data)


# Create a population
population2 = Population(size=population_size, num_nodes=tree_size, file_name='../data/problem_0.npz')

# Evaluate the population's fitness
population2.evaluate_fitness(file_name)

# Get the best individual
best_individual = population2.get_best_individual()
print("Best Fitness Value:", best_individual.fitness(file_name))
print("mse percentage", mse_percentage(best_individual.fitness(file_name), '../data/problem_0.npz'))
print("Best Individual:", best_individual)

worst_individual = population2.get_worst_individual()
print("Worst Fitness Value:", worst_individual.fitness(file_name))
print("mse percentage", mse_percentage(worst_individual.fitness(file_name), '../data/problem_0.npz'))
population2.individuals.remove(worst_individual)

In [43]:
def create_generation(current_population, file_name, crossover_rate=0.8, elite_size = 3):
    """
    Create a new generation of individuals using mutation and crossover.

    :param current_population: The current population of individuals.
    :param file_name: The file name for fitness calculations.
    :param crossover_rate: The probability of performing crossover.
    :param elite_size: The number of top individuals to retain.
    :return: A new Population object representing the next generation.
    """
    # Sort population by fitness and copy elites
    sorted_population = sorted(current_population.individuals, key=lambda ind: ind.fitness(file_name))
    new_generation = (sorted_population[:elite_size])
    
    # Create the rest of the new generation
    while len(new_generation) < len(current_population.individuals):        
        # Perform crossover with probability `crossover_rate`
        if random.random() < crossover_rate:
            # Perform tournament selection to choose parents
            parent1 = current_population.tournament_selection()
            parent2 = current_population.tournament_selection()
            while (parent1 == parent2):
                parent2 = current_population.tournament_selection()
                
            child1, child2 = subtree_crossover(parent1, parent2)
            while (np.isnan(child1.fitness(file_name)) 
                    or np.isinf(child1.fitness(file_name))
                    or np.isnan(child2.fitness(file_name)) 
                    or np.isinf(child2.fitness(file_name))):
                child1, child2 = subtree_crossover(parent1, parent2)
            
            if len(new_generation) < len(current_population.individuals):
                new_generation.append(child1)
            if len(new_generation) < len(current_population.individuals):
                new_generation.append(child2)
        
        else:
            # Perform mutation
            selected_individual = current_population.tournament_selection()
            mutated = subtree_mutation(selected_individual, file_name)
            while (np.isnan(mutated.fitness(file_name)) 
                    or np.isinf(mutated.fitness(file_name))):
                mutated = subtree_mutation(selected_individual, file_name)
            if len(new_generation) < len(current_population.individuals):
                new_generation.append(mutated)

    new_population = Population(size=0, num_nodes=tree_size, file_name=file_name, individuals=new_generation)

    return new_population

In [ ]:
#parameters
file_name = '../data/problem_0.npz'
population_size = 100
elite_size = 3
tree_size = 10
num_generations = 100

current_population = Population(size=population_size, num_nodes=tree_size, file_name=file_name)

best_individual = current_population.get_best_individual()
print("Initial Best Individual:", best_individual)
print("Initial Best MSE Value:", best_individual.fitness(file_name))
print("Initial MSE percentage", mse_percentage(best_individual.fitness(file_name), file_name))

for generation in range(num_generations):
    new_population = create_generation(current_population, file_name, elite_size=elite_size)
            
    # for ind in new_population.individuals:
    #     print(str(ind) + ":" + str(ind.fitness_value))
    
    new_best_individual = new_population.get_best_individual()
    print(f"Generation {generation+1}")
    print("Best Individual:", new_best_individual)
    print("Best MSE Value:", new_best_individual.fitness(file_name))
    current_population.individuals = new_population.individuals[:]
            
print("MSE percentage", mse_percentage(new_population.get_best_individual().fitness_value, file_name))